In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings 
warnings.filterwarnings('ignore')

import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss

In [ ]:
pip install scale

In [2]:
def get_outlier(df=None, column=None, weight=1.5):
    import numpy as np
    age_group_list = [20,30,40,50,60,70]
    outlier_index_set = set()
    
    
    for age_group in age_group_list:
        # fraud에 해당하는 column 데이터만 추출, 1/4 분위와 3/4 분위 지점을 np.percentile로 구함. 
        fraud = df[df['연령대_1']==age_group][column]
        quantile_25 = np.percentile(fraud.values, 25)
        quantile_75 = np.percentile(fraud.values, 75)
        iqr = quantile_75 - quantile_25
        iqr_weight = iqr * weight
        lowest_val = quantile_25 - iqr_weight
        highest_val = quantile_75 + iqr_weight
        # 최대값 보다 크거나, 최소값 보다 작은 값을 아웃라이어로 설정하고 DataFrame index 반환. 
        outlier_index_set = outlier_index_set | set(fraud[(fraud < lowest_val) | (fraud > highest_val)].index)
        
    return outlier_index_set

def remove_outlier(df=None,column_list=None,weight=1.5):
    outlier_index_set = set()
    
    for column_name in column_list:
        outlier_index = get_outlier(df=df, column=column_name, weight=weight)
        outlier_index_set = outlier_index_set | outlier_index
        
    df.drop(list(outlier_index_set), inplace=True)
    df.reset_index(drop=True, inplace=True)


## 통신데이터 컬럼 드랍 및 데이터 프레임 만들기

In [3]:
for m in tqdm(range(1,10),desc='월단위 진행'):
    globals()["ts{}".format(m)] = pd.read_csv(f'./data/2022.{m}월_29개 통신정보.csv',encoding='cp949')
    try : 
        globals()["ts{}".format(m)].drop(columns=['행정동코드','행정동','총인구수','1인가구수'],inplace=True)    
    except:
        globals()["ts{}".format(m)].drop(columns=['행정동코드','행정동명','총인구','1인가구수'],inplace=True)  
    globals()["ts{}".format(m)].drop(columns=globals()["ts{}".format(m)].filter(like='4분위수'),inplace=True)
    globals()["ts{}".format(m)].drop(columns=globals()["ts{}".format(m)].filter(like='인구'),inplace=True) #미추정
    
    # 데이터가 평균값을 제시하고 그 외 4분위 관련 수치를 따로 제공해주고 있음 -> 이미 전처리가 되어있는 거 같음..?
    globals()["ts{}".format(m)]['연령대_1'] = globals()["ts{}".format(m)]['연령대']//10*10
    globals()["ts{}".format(m)]['월'] = int(m)
    fix = globals()["ts{}".format(m)][['월','연령대','연령대_1','자치구','성별']]
    uni = ['상주지','출근','근무','소액결재','연체','카카오톡','SNS','통화','문자','데이터','이동','체류','지하철','게임','금융','쇼핑','동영상','유튜브','넷플릭스','배달']
    
    for col in tqdm(uni,desc=f'{m}월'):
        globals()["{}_{}".format(col,m)] = pd.concat([fix,globals()["ts{}".format(m)].filter(like=col)],axis=1)    
        
        
for col in tqdm(uni,desc='카테고리별로 합치기'):
    globals()["{}".format(col)] = pd.DataFrame()
    for m in range(1,10):
        globals()["이동_{}".format(m)].drop(columns= globals()["이동_{}".format(m)].filter(like='지하철'),inplace=True)
        globals()["{}".format(col)]= pd.concat([globals()["{}".format(col)],globals()["{}_{}".format(col,m)]])

월단위 진행:   0%|          | 0/9 [00:00<?, ?it/s]

1월:   0%|          | 0/20 [00:00<?, ?it/s]

2월:   0%|          | 0/20 [00:00<?, ?it/s]

3월:   0%|          | 0/20 [00:00<?, ?it/s]

4월:   0%|          | 0/20 [00:00<?, ?it/s]

...

6월:   0%|          | 0/20 [00:00<?, ?it/s]

7월:   0%|          | 0/20 [00:00<?, ?it/s]

8월:   0%|          | 0/20 [00:00<?, ?it/s]

9월:   0%|          | 0/20 [00:00<?, ?it/s]

카테고리별로 합치기:   0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
df = pd.concat([ts1,ts2,ts3,ts4,ts5,ts6,ts7,ts8,ts9],axis=0,ignore_index=True)


df

,자치구,성별,연령대,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,소액결재 사용횟수 평균,소액결재 사용금액 평균,최근 3개월 내 요금 연체 비율,...,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수,연령대_1,월
0,종로구,1,20,2.6,3.8,56.6,514.9,1.7,7333.3,4.81,...,82.8,72.1,28.0,0.3,-0.1,10.3,2.5,1.7,20,1
1,종로구,1,25,2.3,3.7,50.3,527.0,2.3,9486.1,12.50,...,109.3,103.6,31.2,0.2,-0.1,13.9,3.0,1.7,20,1
2,종로구,1,30,2.2,3.1,57.4,532.4,2.7,7071.4,9.88,...,130.2,121.7,30.1,0.1,-0.1,13.8,3.2,1.7,30,1
3,종로구,1,35,2.0,2.7,61.6,540.8,2.4,7072.9,11.95,...,123.2,121.4,28.1,0.1,-0.2,11.2,3.3,1.7,30,1
4,종로구,1,40,1.9,2.6,69.3,554.7,2.8,5655.2,4.62,...,122.2,118.0,28.9,0.0,-0.2,9.3,3.1,1.7,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91579,강동구,2,55,1.5,2.7,56.7,502.5,2.3,6623.8,6.68,...,61.5,96.0,15.1,-0.1,-0.6,9.6,3.6,1.7,50,9
91580,강동구,2,60,1.5,2.8,60.8,486.8,2.6,6711.4,6.55,...,45.5,68.0,13.9,-0.1,-0.6,8.9,3.6,1.7,60,9
91581,강동구,2,65,1.4,2.8,60.1,451.1,2.7,6420.3,3.86,...,36.6,52.6,12.5,-0.2,-0.6,7.6,4.2,1.7,60,9
91582,강동구,2,70,1.3,1.4,20.0,520.0,2.8,7051.3,4.49,...,26.2,36.4,10.2,-0.3,-0.6,5.2,0.0,1.7,70,9


In [6]:
df = df[['자치구', '성별', '연령대', '연령대_1', '월', '야간상주지 변경횟수 평균',
       '주간상주지 변경횟수 평균', '평균 출근 소요시간 평균', '평균 근무시간 평균', '소액결재 사용횟수 평균',
       '소액결재 사용금액 평균', '최근 3개월 내 요금 연체 비율', 'SNS 사용횟수', '평균 통화량', '평균 문자량',
       '평균 통화대상자 수', '평균 문자대상자 수', '데이터 사용량', '평일 총 이동 횟수', '휴일 총 이동 횟수 평균',
       '집 추정 위치 평일 총 체류시간', '집 추정 위치 휴일 총 체류시간', '평일 총 이동 거리 합계',
       '휴일 총 이동 거리 합계', '지하철이동일수 합계', '게임 서비스 사용일수', '금융 서비스 사용일수',
       '쇼핑 서비스 사용일수', '동영상/방송 서비스 사용일수', '유튜브 사용일수', '넷플릭스 사용일수',
       '배달 서비스 사용일수', '배달_브랜드 서비스 사용일수', '배달_식재료 서비스 사용일수']]
# 사용할 컬럼만 뽑아 쓰세요~

In [7]:
col_lst = df.columns[5:] #col_lst는 수치형 데이터만 담으세여~
col_= [x for x in col_lst ]
# col_

In [8]:
import pandas as pd

remove_outlier(df,col_)
pd.to_

,자치구,성별,연령대,연령대_1,월,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,소액결재 사용횟수 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,종로구,1,25,20,1,2.3,3.7,50.3,527.0,2.3,...,13.4,66.1,109.3,103.6,31.2,0.2,-0.1,13.9,3.0,1.7
1,종로구,1,30,30,1,2.2,3.1,57.4,532.4,2.7,...,13.4,57.5,130.2,121.7,30.1,0.1,-0.1,13.8,3.2,1.7
2,종로구,1,35,30,1,2.0,2.7,61.6,540.8,2.4,...,13.6,51.7,123.2,121.4,28.1,0.1,-0.2,11.2,3.3,1.7
3,종로구,1,40,40,1,1.9,2.6,69.3,554.7,2.8,...,13.6,47.6,122.2,118.0,28.9,0.0,-0.2,9.3,3.1,1.7
4,종로구,1,45,40,1,1.8,2.5,71.1,530.8,2.5,...,13.8,50.7,95.3,115.6,24.9,-0.1,-0.3,8.5,3.0,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61483,강동구,2,50,50,9,1.5,2.7,62.2,508.7,2.3,...,7.7,40.7,81.9,134.6,18.7,-0.1,-0.5,10.8,3.1,1.7
61484,강동구,2,55,50,9,1.5,2.7,56.7,502.5,2.3,...,7.3,40.7,61.5,96.0,15.1,-0.1,-0.6,9.6,3.6,1.7
61485,강동구,2,60,60,9,1.5,2.8,60.8,486.8,2.6,...,6.7,37.2,45.5,68.0,13.9,-0.1,-0.6,8.9,3.6,1.7
61486,강동구,2,65,60,9,1.4,2.8,60.1,451.1,2.7,...,6.2,36.5,36.6,52.6,12.5,-0.2,-0.6,7.6,4.2,1.7


In [9]:
y

NameError: name 'y' is not defined

In [ ]:
모델의 의의 : 해당 컬럼에서 높게 차지는 연령대의 변화 
예_ 넷플릭스 주 연령층의 변화 추세 -> 주 타겟층 설정시 도움이 되도록.
1인가구에 포커싱하자 -> 